In [7]:
#pip install keras

Note: you may need to restart the kernel to use updated packages.


In [6]:
#pip install tensorflow_hub

   ---------------------------------------- 0.0/85.4 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/85.4 kB ? eta -:--:--
   -------------------------------------- - 81.9/85.4 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 85.4/85.4 kB 963.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
#pip install tensorflow

In [120]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score

In [9]:
# Use forward slashes or double backslashes
file_path = "train.csv"

# Read the CSV file
df = pd.read_csv(file_path)

In [3]:
df.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [23]:
df["text"]

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [17]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print("module %s loaded" % module_url)


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [24]:
def embed(input):
    return model([tf.constant(input)]).numpy()

In [26]:
#get an embbeding that is a numpy
good = embed("this is good")

In [13]:
best = embed("this is best")

In [27]:
bad = embed("this is bad")
worst = embed("this is worst")

In [28]:
#distance between words
np.linalg.norm(best-good)

0.68958265

In [29]:
np.linalg.norm(bad-worst)

0.8577287

In [30]:
man = embed("man")
woman = embed("woman")
king = embed("king")
queen = embed("queen")

In [31]:
np.linalg.norm(king-queen)

0.9071169

In [32]:
np.linalg.norm(man-queen)

1.129361

In [33]:
np.linalg.norm(woman-queen)

1.0875505

In [39]:
ax = [embed(x) for x in df["text"]]

In [40]:
data = np.concatenate(ax)

In [42]:
data.shape

(7613, 512)

In [48]:
t = df["target"].to_numpy()

In [54]:

x_train, x_valid, y_train, y_valid = train_test_split(data, t, test_size=0.3, random_state=29, stratify = t)

In [55]:
x_train.shape

(5329, 512)

In [60]:
x_train[:10]

array([[ 0.04022318,  0.04670988,  0.02312946, ...,  0.03032068,
         0.000341  ,  0.01057979],
       [-0.00919163, -0.01421224, -0.06014035, ...,  0.04734573,
        -0.07046755, -0.08304105],
       [ 0.04523318, -0.05169467, -0.0527    , ...,  0.00758356,
        -0.03641033, -0.08740543],
       ...,
       [ 0.04859615,  0.02199277, -0.03602878, ..., -0.00785076,
        -0.05651842,  0.03602435],
       [-0.03461031, -0.08691164, -0.02851554, ..., -0.04314409,
         0.0971523 , -0.0127969 ],
       [ 0.01065905, -0.0337695 , -0.00143959, ..., -0.03229476,
        -0.07440851, -0.02477588]], dtype=float32)

In [77]:
# continue with projection (PCA) to 2 dimensions (there are too many features, they need to be projected to be visualised)
projector = PCA(n_components=3, whiten=False)

In [78]:
# build projection in train data
projection = projector.fit_transform(x_train)

In [79]:
projection.shape

(5329, 3)

In [87]:
"""fig = px.scatter_3d(x=projection[:,0], y=projection[:,1], z=projection[:,2],
color=y_train)
fig.show()
#too many points, too slow
"""

'fig = px.scatter_3d(x=projection[:,0], y=projection[:,1], z=projection[:,2],\ncolor=y_train)\nfig.show()\n#too many points, too slow\n'

In [95]:
fig = px.scatter_3d(x=projection[::10,0], y=projection[::10,1], z=projection[::10,2],
color=y_train[::10], opacity=0.7)
fig.show()
#take every 10th instance

In [97]:
forest = RandomForestClassifier(n_estimators=100, random_state=100)

In [98]:
forest.fit(x_train, y_train)

RandomForestClassifier(random_state=100)

In [112]:
forest.predict_proba?


Signature: forest.predict_proba(X)
Docstring:
Predict class probabilities for X.

The predicted class probabilities of an input sample are computed as
the mean predicted class probabilities of the trees in the forest.
The class probability of a single tree is the fraction of samples of
the same class in a leaf.

Parameters
----------
X : {array-like, sparse matrix} of shape (n_samples, n_features)
    The input samples. Internally, its dtype will be converted to
    ``dtype=np.float32``. If a sparse matrix is provided, it will be
    converted into a sparse ``csr_matrix``.

Returns
-------
p : ndarray of shape (n_samples, n_classes), or a list of such arrays
    The class probabilities of the input samples. The order of the
    classes corresponds to that in the attribute :term:`classes_`.
File:      c:\users\alex\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local-packages\python311\site-packages\sklearn\ensemble\_forest.py
Type:      method

In [99]:
pred = forest.predict(x_valid)

In [100]:
pred.shape

(2284,)

In [101]:
pred[:10]
#predicted labels

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0], dtype=int64)

In [102]:
accuracy_percent=100*np.sum(pred==y_valid)/len(y_valid)
accuracy_percent

80.47285464098074

In [108]:
accuracy_score(y_valid,pred)

0.8047285464098074

In [109]:
roc_auc_score(y_valid,pred)
#roc less than accuracy? What does it mean?

0.7874175724021175

In [110]:
precision_recall_fscore_support(y_valid,pred)

(array([0.78283828, 0.84785436]),
 array([0.91020721, 0.66462793]),
 array([0.84173172, 0.74514286]),
 array([1303,  981], dtype=int64))

In [113]:
prob = forest.predict_proba(x_valid)

In [114]:
prob[:10]

array([[0.56      , 0.44      ],
       [0.42      , 0.58      ],
       [0.9       , 0.1       ],
       [0.53      , 0.47      ],
       [0.52333333, 0.47666667],
       [0.83      , 0.17      ],
       [0.37      , 0.63      ],
       [0.75333333, 0.24666667],
       [0.13      , 0.87      ],
       [0.75      , 0.25      ]])

In [115]:
pred[:10]

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0], dtype=int64)

In [117]:
prob_2 = prob[:,1]

In [121]:
prob_2[:10]

array([0.44      , 0.58      , 0.1       , 0.47      , 0.47666667,
       0.17      , 0.63      , 0.24666667, 0.87      , 0.25      ])

In [123]:
threshold = 0.8

In [124]:
precision_score(y_valid,prob_2 > threshold)
# we maximize precision to ensure no false positives ("not calling the firemen for nothing")

0.9919678714859438

In [125]:
recall_score(y_valid, prob_2 > threshold)
# it only gets a quarter of the accidents sent in twitter.

0.25178389398572887

In [126]:
accuracy_score(y_valid, prob_2 > threshold)
# affected easily by many factors. As a rule of thumb, not useful by itself.

0.6777583187390543